In [ ]:
from pathlib import Path
import keras
from keras.preprocessing import image
import numpy as np

In [ ]:
f = Path('model_structure.json')
model_structure = f.read_text()

model = keras.model_from_json(model_structure)

model.load_weights('model_weights.h5')

In [ ]:
- Load an image file to test, resizing it to 32x32 pixels (as required by this model)
- Then convert the image to a numpy array (and normalise it)

img= image.load_img('frog.png', target_size=(500,500))

image_to_test = image.img_to_array(img) / 255

In [ ]:
Since Keras expects a list of images, not a single image, we add a fourth dimension to the image's array object

list_of_images = np.expand_dims(image_to_test, axis=0)

In [ ]:
Now we make a prediction using the model
- Results contains the list of results for images we pass in.
- Since we are only testing one image, we only need to check the first result.

results = model.predict(list_of_images)  
single_result = results[0]

In [ ]:
We will get a likelihood score for both possible classifications 
- Now we can find out which class had the highest score.

labels = ['Normal', 'Pneumonia']
most_likely_class_index = int(np.argmax(single_result))
class_likelihood = single_result [most_likely_class_index]
label = labels[most_likely_class_index]
print("This is image is a {} - Likelihood: {:2f}".format(label, class_likelihood))

In [ ]:
most_likely_class_index = int(np.argmax(single_result))
class_likelihood = single_result [most_likely_class_index]
class_label = class_labels[most_likely_class_index]
print("This is image is a {} - Likelihood: {:2f}".format(class_label, class_likelihood))

most_likely_class_index = int(np.argmax(single_result))
class_likelihood = single_result [most_likely_class_index]
class_label = class_labels[most_likely_class_index]
print("This is image is a {} - Likelihood: {:2f}".format(class_label, class_likelihood))

In [ ]:
img= image.load_img('frog.png', target_size=(32,32))

image_to_test = image.img_to_array(img) / 255

Since Keras expects a list of images, not a single image, we add a fourth dimension to the image's array object

In [ ]:
list_of_images = np.expand_dims(image_to_test, axis=0)

Now we make a prediction using the model
- Results contains the list of results for images we pass in.
- Since we are only testing one image, we only need to check the first result.

In [ ]:
results = model.predict(list_of_images)  
single_result = results[0]

We will get a likelihood score for all 10 possible classes. 
- Now we can find out which class had the highest score.

In [ ]:
most_likely_class_index = int(np.argmax(single_result))
class_likelihood = single_result [most_likely_class_index]
class_label = class_labels[most_likely_class_index]
print("This is image is a {} - Likelihood: {:2f}".format(class_label, class_likelihood))